In [53]:
import random
import copy
import numpy as np
import pandas as pd
from datetime import datetime
from collections import defaultdict

# Fungsi Bantu
def time_to_minutes(t):
    try:
        dt = datetime.strptime(t, "%H:%M:%S")
    except ValueError:
        dt = datetime.strptime(t, "%H:%M")
    return dt.hour * 60 + dt.minute

# Inisialisasi Data
dosen_df = pd.read_csv('data_skripsi_dosen.csv')
mk_genap_df = pd.read_csv('data_skripsi_mk_genap.csv')
data_dosen_df = pd.read_csv('data_skripsi_data_dosen.csv')
hari_df = pd.read_csv('data_skripsi_hari.csv')
ruang_df = pd.read_csv('data_skripsi_ruang.csv')
jam_df = pd.read_csv('data_skripsi_jam.csv')

# Urutkan jam_df sebelum generate slot
jam_df = jam_df.sort_values('id_jam')

merged_df = pd.merge(
    pd.merge(data_dosen_df, dosen_df, on='id_dosen'),
    mk_genap_df, on='id_mk_genap'
)

Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    

    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [54]:
# @title Preprosessing
hari_list = hari_df['nama_hari'].tolist()
ruang_list = ruang_df['nama_ruang'].tolist()
jam_list = jam_df[['id_jam', 'jam_awal', 'jam_akhir']].to_dict('records')
mata_kuliah_list = mk_genap_df.set_index('id_mk_genap').to_dict('index')

def slot_generator():
    wolf = []
    id_counter = 1
    for hari in hari_df['nama_hari']:
        for ruang in ruang_df['nama_ruang']:
            for jam in jam_df.itertuples():
                wolf.append({
                    "id_slot": id_counter,
                    "mata_kuliah": None,
                    "dosen": None,
                    "ruang": ruang,
                    "hari": hari,
                    "jam_mulai": jam.jam_awal,
                    "jam_selesai": jam.jam_akhir,
                    "kelas": None,
                    "sks": None,
                    "metode": None
                })
                id_counter += 1
    return wolf

slots = slot_generator()
print("5 slot pertama:")
for slot in slots[:5]:
    print(slot)
    
print("\n5 slot terakhir:")
for slot in slots[-5:]:
    print(slot)

5 slot pertama:
{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}

5 slot terakhir:
{'id_slot': 500, 'mata_kuliah': None, 'dosen': N

In [55]:
# Fungsi utama: buat jadwal acak
def create_random_schedule(ordered_courses_df=None):
    schedule = slot_generator()
    
    # Gunakan parameter jika disediakan, acak jika tidak
    if ordered_courses_df is None:
        merged_shuffled = merged_df.sample(frac=1).iterrows()
    else:
        merged_shuffled = ordered_courses_df.iterrows()
    
    # Tracking alokasi, meskipun pengecekan overlap tidak digunakan, tetap disimpan sebagai referensi
    room_allocations = defaultdict(list)  # key: (ruang, hari) -> list of (start, end)
    teacher_allocations = defaultdict(list)  # key: (dosen, hari) -> list of (start, end)
    class_allocations = defaultdict(list)  # key: (kelas, hari) -> list of (start, end)
    
    for _, row in merged_shuffled:
        mata_kuliah = row['nama_mk_genap']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        metode = row['metode']
        
        possible_positions = list(range(len(schedule) - sks + 1))
        random.shuffle(possible_positions)
        
        candidate_blocks = []
        for i in possible_positions:
            block = schedule[i:i+sks]
            
            # Pastikan semua slot kosong dan berada di hari yang sama
            if not all(slot['mata_kuliah'] is None for slot in block) or not all(slot['hari'] == block[0]['hari'] for slot in block):
                continue
                
            # Untuk offline, cek kesamaan ruangan
            if not all(slot['ruang'] == block[0]['ruang'] for slot in block):
                continue
                
            # Pengecekan waktu berurutan dihapus, sehingga blok diperbolehkan memiliki gap waktu
            hari = block[0]['hari']
            ruang = block[0]['ruang']
            time_block = (time_to_minutes(block[0]['jam_mulai']), time_to_minutes(block[-1]['jam_selesai']))
            
            # Pengecekan overlap dihapus, jadi langsung tambahkan blok kandidat
            kelas_already = len(class_allocations[(kelas, hari)]) > 0
            candidate_blocks.append((block, time_block, kelas_already))
        
        # Pemilihan blok kandidat: ambil blok pertama jika ada kandidat
        if candidate_blocks:
            selected_block = candidate_blocks[0][0]
            
            for slot in selected_block:
                slot.update({
                    "mata_kuliah": mata_kuliah,
                    "dosen": dosen,
                    "kelas": kelas,
                    "sks": sks,
                    "metode": metode
                    # Tidak melakukan update ruang karena sudah terisi dari slot generator
                })
                
            hari = selected_block[0]['hari']
            ruang = selected_block[0]['ruang']
            time_block = (time_to_minutes(selected_block[0]['jam_mulai']),
                         time_to_minutes(selected_block[-1]['jam_selesai']))
                         
            room_allocations[(ruang, hari)].append(time_block)
            teacher_allocations[(dosen, hari)].append(time_block)
            class_allocations[(kelas, hari)].append(time_block)
        else:
            print(f"Gagal menempatkan: {kelas} - {mata_kuliah} - {dosen}")
    
    return schedule

# Contoh penggunaan:
population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

# Uncomment untuk menampilkan jadwal
i=0
for schedule in population:
    for slot in schedule:
        if slot['mata_kuliah'] is not None:
            i+=1
        # print(slot)
    if i == merged_df.sum()['sks']:
        print("Jadwal Sudah Lengkap")
    else:
        print("Jadwal Belum Lengkap")

print("Jumlah slot yang terisi: ", i)


Jadwal Sudah Lengkap
Jumlah slot yang terisi:  282


Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
dosen tidak boleh mengajar mata kuliah/kelas berbeda pada jam yang sama
ruang kelas tidak boleh digunakan lebih dari 1 kelas/dosen
3 sks membutuhkan 3 slot waktu
ruangan antar kelas harus sama
slot waktu harus berurutan

Konstrain ringan: pereferensi dosen
dosen Ardiansyah, Dr., S.T., M.Cs. Tidak ingin kelas sebelum 12:00 PM
dosen Ali Tarmuji, S.T., M.Cs. Tidak ingin ada kelas pada hari Sabtu
dosen Bambang Robiin, S.T., M.T. tidak ingin kelas setelah 12:00 PM
dosen Tedy Setiadi, Drs., M.T. Tidak ingin ada kelas pada hari Sabtu dan Kamis

In [56]:
# def calculate_fitness(schedule):
#     penalty = 0

#     teacher_conflicts = []              # Hard constraint: dosen mengajar dua kelas bersamaan
#     room_conflicts = []                 # Hard constraint: ruangan dipakai lebih dari satu kelas bersamaan
    
#     teacher_preference_conflicts = []   # Soft constraint: preferensi dosen
#     day_conflicts = []             # Soft constraint: hari Sabtu

#     # Tracking hard constraint
#     teacher_intervals = defaultdict(list)
#     room_intervals = defaultdict(list)

#     for slot in schedule:
#         if not slot['mata_kuliah']:
#             continue

#         day = slot['hari']
#         teacher = slot['dosen']
#         room = slot['ruang']
#         kelas = slot['kelas']
#         id_slot = slot['id_slot']
#         start = time_to_minutes(slot['jam_mulai'])
#         end = time_to_minutes(slot['jam_selesai'])
        
#         teacher_intervals[(teacher, day)].append((start, end, id_slot))
#         if room != "Online":
#             room_intervals[(room, day)].append((start, end, id_slot))
        
#         # Soft constraint: preferensi dosen
#         if str(teacher) in ["Ardiansyah, Dr., S.T., M.Cs."]:
#             if start <= 720:
#                 teacher_preference_conflicts.append({
#                     'teacher': teacher,
#                     'id_slot': id_slot,
#                     'jam': slot['jam_mulai'],
#                     'message': "Hanya ingin dijadwalkan pada jam 12 atau lebih"
#                 })
#                 penalty += 0.5

#         # Soft constraint: setiap slot pada hari Sabtu
#         if str(teacher) in ["Tedy Setiadi, Drs., M.T."]:
#             if day == "Sabtu":
#                 day_conflicts.append({
#                     'id_slot': id_slot,
#                     'teacher': teacher,
#                     'kelas': kelas,
#                     'hari': day,
#                     'message': "Kelas tidak ingin dijadwalkan pada hari Sabtu"
#                 })
#                 penalty += 0.5

#     # Hard constraint: Dosen
#     for (teacher, day), intervals in teacher_intervals.items():
#         intervals.sort(key=lambda x: x[0])
#         for i in range(1, len(intervals)):
#             prev = intervals[i-1]
#             curr = intervals[i]
#             if curr[0] < prev[1]:
#                 teacher_conflicts.append({
#                     'teacher': teacher,
#                     'conflict_ids': (prev[2], curr[2])
#                 })
#                 penalty += 1

#     # Hard constraint: Ruangan
#     for (room, day), intervals in room_intervals.items():
#         intervals.sort(key=lambda x: x[0])
#         for i in range(1, len(intervals)):
#             prev = intervals[i-1]
#             curr = intervals[i]
#             if curr[0] < prev[1]:
#                 room_conflicts.append({
#                     'room': room,
#                     'day': day,
#                     'conflict_ids': (prev[2], curr[2]),
#                     'messages': "dipakai lebih dari 1 kelas"
#                 })
#                 penalty += 1
    
#     return {
#         'penalty': penalty,
#         'violations': {
#             'hard_teacher': len(teacher_conflicts),
#             'hard_room': len(room_conflicts),
#             'teacher_preference': len(teacher_preference_conflicts),
#             'day_conflicts': len(day_conflicts)
#         },
#         'teacher_conflicts': teacher_conflicts,
#         'room_conflicts': room_conflicts,
#         'teacher_preference_conflicts': teacher_preference_conflicts,
#         'day_conflicts': day_conflicts
#     }

# # validasi ruangan
# # def compare_room_usage(schedule):
# #     room_usage = defaultdict(list)
    
# #     for slot in schedule:
# #         if not slot['mata_kuliah']:
# #             continue
# #         room = slot['ruang']
# #         usage_info = {
# #             'id_slot': slot['id_slot'],
# #             'dosen': slot['dosen'],
# #             'kelas': slot['kelas'],
# #             'hari': slot['hari'],
# #             'jam_mulai': slot['jam_mulai'],
# #             'jam_selesai': slot['jam_selesai']
# #         }
# #         room_usage[room].append(usage_info)
    
# #     for room in room_usage:
# #         room_usage[room].sort(key=lambda x: (x['hari'], time_to_minutes(x['jam_mulai'])))
# #     return room_usage

# population_size = 1
# population = [create_random_schedule() for _ in range(population_size)]

# for idx, schedule in enumerate(population):
#     fitness = calculate_fitness(schedule)
#     print(f"\nIndividu {idx+1}:")
#     for slot in schedule:
#         print(slot)

#     print(f"\nTotal Penalty: {fitness['penalty']}")
    
#     # validasi konflik dosen (hard constraint)
#     print("\nDetail Konflik Dosen:")
#     if not fitness['teacher_conflicts']:
#         print(" - Tidak ada konflik dosen.")
#     else:
#         for conflict in fitness['teacher_conflicts']:
#             conflict_ids = conflict['conflict_ids']
#             print(f" - {conflict['teacher']}: Konflik pada Slot {conflict_ids[0]} dan {conflict_ids[1]}")
    
#     # validasi ruangan (hard constraint)
#     print("\nValidasi Data Ruangan:")
#     room_conflicts = fitness['room_conflicts']
#     if not room_conflicts:
#         print(" - Data ruangan valid, tidak ada konflik.")
#     else:
#         for conflict in room_conflicts:
#             conflict_ids = conflict['conflict_ids']
#             print(f" - Ruang {conflict['room']} {conflict['message']} : Konflik pada Slot {conflict_ids[0]} dan {conflict_ids[1]}")

#     # Soft constraint: preferensi dosen
#     print("\nSoft Constraint - Preferensi Dosen (Jam 6 sampai 12):")
#     if not fitness['teacher_preference_conflicts']:
#         print(" - Tidak ada konflik preferensi dosen.")
#     else:
#         for conflict in fitness['teacher_preference_conflicts']:
#             print(f" - Slot {conflict['id_slot']} - {conflict['teacher']} (jam mulai: {conflict['jam']}): {conflict['message']}")

#     # Soft constraint: hari Sabtu
#     print("\nSoft Constraint - Preferensi Dosen selain hari sabtu:")
#     if not fitness['day_conflicts']:
#         print(f" - Data valid, tidak ada konflik.")
#     else:
#         for conflict in fitness['day_conflicts']:
#             print(f" - Slot {conflict['id_slot']} - {conflict['teacher']}, Kelas {conflict['kelas']}: {conflict['message']}")

#     # Komparasi Penggunaan Ruangan Berdasarkan Ruangan
#     # print("\nKomparasi Penggunaan Ruangan Berdasarkan Ruangan:")
#     # room_usage = compare_room_usage(schedule)
#     # if not room_usage:
#     #     print(" - Tidak ada data penggunaan ruangan.")
#     # else:
#     #     for room, usages in room_usage.items():
#     #         print(f" - Ruang: {room}")
#     #         for usage in usages:
#     #             print(f"    ID Slot: {usage['id_slot']}, Dosen: {usage['dosen']}, Kelas: {usage['kelas']}, Hari: {usage['hari']}, Jam Mulai: {usage['jam_mulai']}, Jam Selesai: {usage['jam_selesai']}")


In [57]:
# Fungsi Fitness dengan Perbaikan
# Fungsi pembantu: Deteksi konflik pada kumpulan interval waktu
def detect_time_conflicts(intervals):
    conflicts = []
    intervals.sort(key=lambda x: x[0])  # Urutkan berdasarkan waktu mulai
    for i in range(1, len(intervals)):
        # Jika waktu mulai slot saat ini lebih kecil dari waktu selesai slot sebelumnya
        if intervals[i][0] < intervals[i-1][1]:
            conflicts.append((intervals[i-1][2], intervals[i][2]))
    return conflicts

# Fungsi untuk mengambil konfigurasi preferensi dosen
def get_lecturer_preferences():
    return {
        "Ardiansyah, Dr., S.T., M.Cs.": [
            {"type": "time_before", "value": 720}  # Tidak ada kelas sebelum 12:00 PM (720 menit)
        ],
        "Ali Tarmuji, S.T., M.Cs.": [
            {"type": "restricted_day", "value": "sabtu"}  # Tidak ada kelas pada hari Sabtu
        ],
        "Bambang Robiin, S.T., M.T.": [
            {"type": "time_after", "value": 720}  # Tidak ingin kelas setelah 12:00 PM
        ],
        "Tedy Setiadi, Drs., M.T.": [
            {"type": "restricted_day", "value": "sabtu, kamis"}  # Tidak ada kelas pada hari Sabtu atau Kamis
        ]
    }

def collect_conflicts(schedule):
    teacher_intervals = defaultdict(list)
    room_intervals = defaultdict(list)
    conflict_slots = set()  # Menyimpan id_slot yang mengalami konflik
    lecturer_preferences = get_lecturer_preferences()
    
    # Untuk menyimpan konflik preferensi dosen
    preference_conflict_slots = set()
    
    # Kelompokkan slot berdasarkan mata kuliah-dosen-kelas
    course_teacher_class = defaultdict(list)
    
    # Kumpulkan interval dan cek konflik preferensi dosen
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue

        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        slot_id = slot['id_slot']
        dosen = str(slot['dosen'])
        hari = slot['hari'].lower()

        # Kumpulkan interval dosen dan ruangan
        teacher_intervals[(dosen, hari)].append((start, end, slot_id))
        if slot['metode'] != 'Online':
            room_intervals[(slot['ruang'], hari)].append((start, end, slot_id))
        
        # Kelompokkan untuk pengecekan konsistensi ruangan & urutan slot
        key = (slot['mata_kuliah'], slot['dosen'], slot['kelas'])
        course_teacher_class[key].append(slot)
        
        # Cek konflik berdasarkan preferensi dosen (konstrain ringan)
        if dosen in lecturer_preferences:
            for pref in lecturer_preferences[dosen]:
                violated = False
                if pref["type"] == "time_before" and start < pref["value"]:
                    # Ardiansyah: Tidak ingin kelas sebelum 12:00 PM
                    violated = True
                elif pref["type"] == "time_after" and start >= pref["value"]:
                    # Bambang Robiin: Tidak ingin kelas setelah 12:00 PM
                    violated = True
                elif pref["type"] == "restricted_day":
                    # Ali Tarmuji dan Tedy Setiadi: Tidak ingin kelas pada hari tertentu
                    days = [d.strip() for d in pref["value"].split(',')]
                    if hari in days:
                        violated = True
                
                if violated:
                    preference_conflict_slots.add(slot_id)

    # Deteksi konflik dosen (konstrain berat)
    teacher_conflicts = []
    for key, intervals in teacher_intervals.items():
        conflicts = detect_time_conflicts(intervals)
        teacher_conflicts.extend(conflicts)
        for c in conflicts:
            conflict_slots.update(c)

    # Deteksi konflik ruangan (konstrain berat)
    room_conflicts = []
    for key, intervals in room_intervals.items():
        conflicts = detect_time_conflicts(intervals)
        room_conflicts.extend(conflicts)
        for c in conflicts:
            conflict_slots.update(c)

    # Cek konstrain berat: konsistensi ruang & slot harus berurutan
    room_consistency_conflicts = []
    sequence_conflicts = []
    
    for key, slots in course_teacher_class.items():
        # Periksa bahwa untuk kelas dengan SKS > 1, harus memiliki jumlah slot yang benar
        expected_slots = slots[0]['sks']  # Ambil nilai SKS dari slot pertama
        
        # Kelompokkan berdasarkan hari
        days = defaultdict(list)
        for s in slots:
            days[s['hari']].append(s)
        
        for day, day_slots in days.items():
            if len(day_slots) > 1:
                # Periksa konsistensi ruangan per hari
                rooms = {s['ruang'] for s in day_slots}
                if len(rooms) > 1:
                    for s in day_slots:
                        conflict_slots.add(s['id_slot'])
                    room_consistency_conflicts.append({
                        'course_key': key,
                        'slot_ids': [s['id_slot'] for s in day_slots],
                        'hari': day
                    })
                
                # Sortir slot berdasarkan jam mulai
                day_slots.sort(key=lambda x: time_to_minutes(x['jam_mulai']))
                
                # Periksa apakah slot berurutan (konstrain berat)
                for i in range(1, len(day_slots)):
                    curr_end = time_to_minutes(day_slots[i-1]['jam_selesai'])
                    next_start = time_to_minutes(day_slots[i]['jam_mulai'])
                    if curr_end != next_start:  # Jika tidak berurutan
                        conflict_slots.add(day_slots[i-1]['id_slot'])
                        conflict_slots.add(day_slots[i]['id_slot'])
                        sequence_conflicts.append({
                            'course_key': key,
                            'prev_slot': day_slots[i-1]['id_slot'],
                            'next_slot': day_slots[i]['id_slot'],
                            'hari': day
                        })
    
    # Periksa jumlah SKS sesuai
    sks_conflicts = []
    for key, slots in course_teacher_class.items():
        expected_slots = slots[0]['sks']  # Ambil nilai SKS dari slot pertama
        if len(slots) != expected_slots:
            for s in slots:
                conflict_slots.add(s['id_slot'])
            sks_conflicts.append({
                'course_key': key,
                'expected': expected_slots,
                'actual': len(slots)
            })

    return {
        'conflict_slots': conflict_slots,
        'preference_conflict_slots': preference_conflict_slots,
        'teacher_conflicts': teacher_conflicts,  # Konstrain berat (1.0)
        'room_conflicts': room_conflicts,  # Konstrain berat (1.0)
        'room_consistency_conflicts': room_consistency_conflicts,  # Konstrain berat (1.0)
        'sequence_conflicts': sequence_conflicts,  # Konstrain berat (1.0)
        'sks_conflicts': sks_conflicts,  # Konstrain berat (1.0)
    }

def calculate_fitness(schedule):
    conflicts = collect_conflicts(schedule)
    penalty = 0.0

    # Konstrain berat (penalty 1.0)
    penalty += len(conflicts['teacher_conflicts']) * 1.0  # Dosen tidak boleh mengajar mata kuliah berbeda pada jam yang sama
    penalty += len(conflicts['room_conflicts']) * 1.0  # Ruang kelas tidak boleh digunakan lebih dari 1 kelas/dosen
    penalty += len(conflicts['room_consistency_conflicts']) * 1.0  # Ruangan antar kelas harus sama
    penalty += len(conflicts['sequence_conflicts']) * 1.0  # Slot waktu harus berurutan
    penalty += len(conflicts['sks_conflicts']) * 1.0  # 3 SKS membutuhkan 3 slot waktu
    
    # Konstrain ringan (penalty 0.5)
    penalty += len(conflicts['preference_conflict_slots']) * 0.5  # Preferensi dosen
    
    return penalty

# Inisialisasi populasi
population_size = 5
population = [create_random_schedule() for _ in range(population_size)]

# Evaluasi fitness untuk setiap individu
for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    print(f"Total Penalty: {fitness}")


Individu 1:
Total Penalty: 110.0

Individu 2:
Total Penalty: 110.0

Individu 3:
Total Penalty: 102.0

Individu 4:
Total Penalty: 101.5

Individu 5:
Total Penalty: 112.5


In [59]:
import json
import numpy as np
import random
import copy

class GreyWolfOptimizer:
    def __init__(self, population_size=10, max_iterations=100):
        self.population_size = population_size
        self.max_iterations = max_iterations
        
    def optimize(self, fitness_function, create_solution_function, collect_conflicts_func):
        # Inisialisasi populasi
        population = [create_solution_function() for _ in range(self.population_size)]
        
        # Evaluasi fitness awal
        fitness_values = [fitness_function(solution) for solution in population]
        
        # Track solusi terbaik
        best_solution = None
        best_fitness = float('inf')
        prev_best_fitness = float('inf')
        
        # Parameter GWO
        a_start = 2.0
        
        # Probabilitas default
        p_alpha = 0.5
        p_beta = 0.3
        p_delta = 0.2
        
        # Iterasi optimasi
        for iteration in range(self.max_iterations):
            # Urutkan solusi berdasarkan fitness (ascending)
            sorted_indices = np.argsort(fitness_values)
            
            # Identifikasi Alpha, Beta, dan Delta
            alpha_idx = sorted_indices[0]
            beta_idx = sorted_indices[1]
            delta_idx = sorted_indices[2]
            
            alpha = population[alpha_idx]
            beta = population[beta_idx] 
            delta = population[delta_idx]
            
            alpha_fitness = fitness_values[alpha_idx]
            
            # Simpan solusi terbaik
            if alpha_fitness < best_fitness:
                best_fitness = alpha_fitness
                best_solution = copy.deepcopy(alpha)
            
            print(f"Iterasi {iteration+1}/{self.max_iterations} - Best Fitness: {best_fitness}")
            
            # Update posisi setiap serigala
            new_population = []
            
            for i in range(self.population_size):
                # Update parameter a
                a = a_start - iteration * (a_start / self.max_iterations)
                a = a * (0.8 + 0.4 * random.random())
                
                # Random restart with small probability
                if random.random() < 0.05:
                    new_solution = create_solution_function()
                else:
                    # Buat solusi baru berdasarkan alpha, beta, dan delta
                    new_solution = self.update_position(
                        population[i], alpha, beta, delta, 
                        a, create_solution_function, fitness_function,
                        p_alpha, p_beta, p_delta
                    )
                
                new_fitness = fitness_function(new_solution)
                
                # Tambahkan ke populasi baru
                new_population.append(new_solution)
                fitness_values[i] = new_fitness
                
            # Update populasi
            population = new_population
            prev_best_fitness = best_fitness
            
        # Analisis konflik final pada solusi terbaik
        final_conflicts = collect_conflicts_func(best_solution)
        
        print("\nHasil Optimasi Final:")
        print(f"Best Fitness: {best_fitness}")
        print("Detail Konflik pada Solusi Terbaik:")
        print(f"  Konflik Dosen: {len(final_conflicts['teacher_conflicts'])}")
        if final_conflicts['teacher_conflicts']:
            print("    Detail konflik dosen:")
            for conflict in final_conflicts['teacher_conflicts'][:5]:
                slot1 = next((s for s in best_solution if s['id_slot'] == conflict[0]), None)
                slot2 = next((s for s in best_solution if s['id_slot'] == conflict[1]), None)
                if slot1 and slot2:
                    print(f"      Dosen {slot1['dosen']} memiliki jadwal bentrok: {slot1['mata_kuliah']} dan {slot2['mata_kuliah']} pada {slot1['hari']} {slot1['jam_mulai']}")
        
        print(f"  Konflik Ruangan: {len(final_conflicts['room_conflicts'])}")
        if final_conflicts['room_conflicts']:
            print("    Detail konflik ruangan:")
            for conflict in final_conflicts['room_conflicts'][:5]:
                slot1 = next((s for s in best_solution if s['id_slot'] == conflict[0]), None)
                slot2 = next((s for s in best_solution if s['id_slot'] == conflict[1]), None)
                if slot1 and slot2:
                    print(f"      Ruangan {slot1['ruang']} digunakan ganda: {slot1['mata_kuliah']} dan {slot2['mata_kuliah']} pada {slot1['hari']} {slot1['jam_mulai']}")
        
        print(f"  Konflik Konsistensi Ruangan: {len(final_conflicts['room_consistency_conflicts'])}")
        print(f"  Konflik Urutan Slot: {len(final_conflicts['sequence_conflicts'])}")
        print(f"  Konflik SKS: {len(final_conflicts['sks_conflicts'])}")
        print(f"  Konflik Preferensi Dosen: {len(final_conflicts['preference_conflict_slots'])}")
        if final_conflicts['preference_conflict_slots']:
            print("    Detail konflik preferensi dosen:")
            preference_slots = [s for s in best_solution if s['id_slot'] in final_conflicts['preference_conflict_slots']]
            for i, slot in enumerate(preference_slots[:5]):
                print(f"      {slot['dosen']} dijadwalkan pada {slot['hari']} {slot['jam_mulai']} (melanggar preferensi)")
            
        # Count scheduled courses
        scheduled_slots = sum(1 for slot in best_solution if slot['mata_kuliah'] is not None)
        print(f"\nTotal slot terisi: {scheduled_slots}")
            
        return best_solution, best_fitness
    
    def update_position(self, current_solution, alpha, beta, delta, a, create_solution_function, fitness_function, p_alpha=0.5, p_beta=0.3, p_delta=0.2):
        """
        Update posisi serigala berdasarkan posisi Alpha, Beta, dan Delta
        dengan perbaikan untuk memastikan semua mata kuliah tetap terjadwal
        """
        new_solution = copy.deepcopy(current_solution)
        
        # STEP 1: Kumpulkan semua mata kuliah yang terjadwal di solusi saat ini
        current_courses = []
        for slot in current_solution:
            if slot['mata_kuliah'] is not None:
                course_key = (slot['mata_kuliah'], slot['dosen'], slot['kelas'])
                if not any(c['key'] == course_key for c in current_courses):
                    current_courses.append({
                        'key': course_key,
                        'mata_kuliah': slot['mata_kuliah'],
                        'dosen': slot['dosen'],
                        'kelas': slot['kelas'],
                        'sks': slot['sks'],
                        'metode': slot['metode']
                    })
        
        # STEP 2: Pilih slot secara acak untuk diubah (simulasi eksplorasi)
        filled_slots = [i for i, slot in enumerate(new_solution) if slot['mata_kuliah'] is not None]
        num_slots_to_change = max(1, int(len(filled_slots) * (a/2)))
        
        # Add randomness to overcome local optima
        if random.random() < 0.1:  # 10% chance
            num_slots_to_change = max(2, int(len(filled_slots) * 0.3))
        
        slots_to_change = random.sample(filled_slots, min(num_slots_to_change, len(filled_slots)))
        
        # STEP 3: Reset slot yang dipilih dan catat mata kuliah yang terkena
        affected_courses = set()
        for idx in slots_to_change:
            if new_solution[idx]['mata_kuliah'] is not None:
                course_key = (new_solution[idx]['mata_kuliah'], new_solution[idx]['dosen'], new_solution[idx]['kelas'])
                affected_courses.add(course_key)
                
            # Reset slot
            new_solution[idx].update({
                "mata_kuliah": None,
                "dosen": None,
                "kelas": None,
                "sks": None,
                "metode": None
            })
        
        # STEP 4: Reset slot yang berkaitan dengan mata kuliah yang terkena
        for i, slot in enumerate(new_solution):
            if slot['mata_kuliah'] is not None:
                course_key = (slot['mata_kuliah'], slot['dosen'], slot['kelas'])
                if course_key in affected_courses:
                    new_solution[i].update({
                        "mata_kuliah": None,
                        "dosen": None,
                        "kelas": None,
                        "sks": None,
                        "metode": None
                    })
        
        # STEP 5: Gabungkan mata kuliah dari Alpha, Beta, dan Delta dengan probabilitas
        courses_to_reschedule = []
        used_keys = set()
        
        # Function to add course with checking to avoid duplicates
        def add_course_if_new(course_data, source_wolf, probability):
            key = (course_data['mata_kuliah'], course_data['dosen'], course_data['kelas'])
            if key not in used_keys and random.random() < probability:
                used_keys.add(key)
                courses_to_reschedule.append(course_data)
                return True
            return False
        
        # Collect courses from wolves
        for slot in alpha:
            if slot['mata_kuliah'] is not None:
                course_data = {
                    'mata_kuliah': slot['mata_kuliah'],
                    'dosen': slot['dosen'],
                    'kelas': slot['kelas'],
                    'sks': slot['sks'],
                    'metode': slot['metode']
                }
                add_course_if_new(course_data, 'alpha', p_alpha)
        
        for slot in beta:
            if slot['mata_kuliah'] is not None:
                course_data = {
                    'mata_kuliah': slot['mata_kuliah'],
                    'dosen': slot['dosen'],
                    'kelas': slot['kelas'],
                    'sks': slot['sks'],
                    'metode': slot['metode']
                }
                add_course_if_new(course_data, 'beta', p_beta)
        
        for slot in delta:
            if slot['mata_kuliah'] is not None:
                course_data = {
                    'mata_kuliah': slot['mata_kuliah'],
                    'dosen': slot['dosen'],
                    'kelas': slot['kelas'],
                    'sks': slot['sks'],
                    'metode': slot['metode']
                }
                add_course_if_new(course_data, 'delta', p_delta)
        
        # PENTING: Tambahkan semua mata kuliah original yang belum terpilih
        for course in current_courses:
            key = course['key']
            if key not in used_keys:
                courses_to_reschedule.append({
                    'mata_kuliah': course['mata_kuliah'],
                    'dosen': course['dosen'],
                    'kelas': course['kelas'],
                    'sks': course['sks'],
                    'metode': course['metode']
                })
                used_keys.add(key)
        
        # Pastikan semua mata kuliah terkena perubahan juga dimasukkan
        for key in affected_courses:
            if key not in used_keys:
                # Cari detail mata kuliah dari current_courses
                course_data = next((c for c in current_courses if c['key'] == key), None)
                if course_data:
                    courses_to_reschedule.append({
                        'mata_kuliah': course_data['mata_kuliah'],
                        'dosen': course_data['dosen'],
                        'kelas': course_data['kelas'],
                        'sks': course_data['sks'],
                        'metode': course_data['metode']
                    })
                    used_keys.add(key)
        
        # STEP 6: Acak urutan kursus dan jadwalkan ulang
        random.shuffle(courses_to_reschedule)
        
        # Jadwalkan ulang semua mata kuliah
        for course in courses_to_reschedule:
            # Coba jadwalkan dengan retry jika gagal
            success = self.schedule_course(new_solution, course)
            retry_count = 0
            
            # Jika gagal, coba beberapa kali lagi dengan slot acak
            while not success and retry_count < 5:
                # Reset beberapa slot acak untuk membuat ruang
                if len(filled_slots) > 0:
                    reset_idx = random.sample(filled_slots, min(5, len(filled_slots)))
                    for idx in reset_idx:
                        new_solution[idx].update({
                            "mata_kuliah": None,
                            "dosen": None,
                            "kelas": None,
                            "sks": None,
                            "metode": None
                        })
                
                # Coba jadwalkan lagi
                success = self.schedule_course(new_solution, course)
                retry_count += 1
        
        # Update filled_slots setelah penjadwalan
        filled_slots = [i for i, slot in enumerate(new_solution) if slot['mata_kuliah'] is not None]
        
        return new_solution
    
    def add_course_if_new(course_data, source_wolf, probability):
            key = (course_data['mata_kuliah'], course_data['dosen'], course_data['kelas'])
            if key not in used_keys and (random.random() < probability or key in affected_courses):
                used_keys.add(key)
                courses_to_reschedule.append(course_data)
                return True
            return False
        
            # Collect courses from wolves with adjusted probabilities
            for slot in alpha:
                if slot['mata_kuliah'] is not None:
                    course_data = {
                        'mata_kuliah': slot['mata_kuliah'],
                        'dosen': slot['dosen'],
                        'kelas': slot['kelas'],
                        'sks': slot['sks'],
                        'metode': slot['metode']
                    }
                    add_course_if_new(course_data, 'alpha', p_alpha)
            
            for slot in beta:
                if slot['mata_kuliah'] is not None:
                    course_data = {
                        'mata_kuliah': slot['mata_kuliah'],
                        'dosen': slot['dosen'],
                        'kelas': slot['kelas'],
                        'sks': slot['sks'],
                        'metode': slot['metode']
                    }
                    add_course_if_new(course_data, 'beta', p_beta)
            
            for slot in delta:
                if slot['mata_kuliah'] is not None:
                    course_data = {
                        'mata_kuliah': slot['mata_kuliah'],
                        'dosen': slot['dosen'],
                        'kelas': slot['kelas'],
                        'sks': slot['sks'],
                        'metode': slot['metode']
                    }
                    add_course_if_new(course_data, 'delta', p_delta)
            
            # STEP 6: Tambahkan semua mata kuliah original yang belum terpilih
            # CRITICAL: Memastikan tidak ada mata kuliah yang hilang
            for course in current_courses:
                key = course['key']
                if key not in used_keys:
                    courses_to_reschedule.append({
                        'mata_kuliah': course['mata_kuliah'],
                        'dosen': course['dosen'],
                        'kelas': course['kelas'],
                        'sks': course['sks'],
                        'metode': course['metode']
                    })
                    used_keys.add(key)
            
            # STEP 7: Acak urutan kursus dan jadwalkan ulang
            random.shuffle(courses_to_reschedule)
            
            # Tambahkan variasi urutan berdasarkan SKS
            # Jadwalkan mata kuliah dengan SKS lebih besar terlebih dahulu
            courses_to_reschedule.sort(key=lambda x: x['sks'], reverse=True)
            
            # Jadwalkan ulang semua mata kuliah
            scheduled_count = 0
            failed_courses = []
            
            for course in courses_to_reschedule:
                # Coba jadwalkan
                success = self.schedule_course(new_solution, course)
                
                if success:
                    scheduled_count += 1
                else:
                    failed_courses.append(course)
            
            # Retry scheduling for failed courses with more aggressive room clearing
            retry_rounds = 0
            max_retries = 3
            
            while failed_courses and retry_rounds < max_retries:
                retry_rounds += 1
                
                # Clear more slots to make room
                if filled_slots:
                    clear_count = min(len(filled_slots)//4 + retry_rounds, len(filled_slots))
                    reset_indices = random.sample([i for i, slot in enumerate(new_solution) 
                                                if slot['mata_kuliah'] is not None], clear_count)
                    
                    for idx in reset_indices:
                        new_solution[idx].update({
                            "mata_kuliah": None,
                            "dosen": None,
                            "kelas": None,
                            "sks": None,
                            "metode": None
                        })
                
                # Try scheduling failed courses again
                still_failed = []
                for course in failed_courses:
                    success = self.schedule_course(new_solution, course)
                    if not success:
                        still_failed.append(course)
                
                failed_courses = still_failed
            
            # Final attempt to schedule any remaining courses
            if failed_courses:
                print(f"  [Warning: {len(failed_courses)} courses couldn't be scheduled initially]")
                # Completely reset the solution and try again if necessary
                temp_solution = copy.deepcopy(new_solution)
                
                # Try clear 50% of the schedule as last resort
                used_slots = [i for i, slot in enumerate(temp_solution) if slot['mata_kuliah'] is not None]
                if used_slots:
                    clear_indices = random.sample(used_slots, len(used_slots)//2)
                    for idx in clear_indices:
                        temp_solution[idx].update({
                            "mata_kuliah": None,
                            "dosen": None,
                            "kelas": None,
                            "sks": None,
                            "metode": None
                        })
                
                # Try one more time to schedule failed courses
                for course in failed_courses:
                    success = self.schedule_course(temp_solution, course)
                    if success:
                        # If succeeded with the temporary solution, adopt it
                        new_solution = temp_solution
                        break
            
            return new_solution

    def schedule_course(self, schedule, course):
        """Menjadwalkan kursus ke dalam slot yang tersedia dan mengembalikan status keberhasilan"""
        mata_kuliah = course['mata_kuliah']
        dosen = course['dosen']
        kelas = course['kelas']
        sks = course['sks']
        metode = course['metode']
        
        # Cari semua kemungkinan posisi
        possible_positions = []
        
        for i in range(len(schedule) - sks + 1):
            block = schedule[i:i+sks]
            
            # Pastikan semua slot kosong dan berada di hari yang sama
            if not all(slot['mata_kuliah'] is None for slot in block) or not all(slot['hari'] == block[0]['hari'] for slot in block):
                continue
            
            # Untuk offline, cek kesamaan ruangan
            if not all(slot['ruang'] == block[0]['ruang'] for slot in block):
                continue
            
            # Pastikan slot berurutan
            valid_sequence = True
            for j in range(1, len(block)):
                prev_end = time_to_minutes(block[j-1]['jam_selesai'])
                curr_start = time_to_minutes(block[j]['jam_mulai'])
                if prev_end != curr_start:
                    valid_sequence = False
                    break
            
            if valid_sequence:
                possible_positions.append(i)
        
        # Jika ada posisi yang tersedia, jadwalkan kursus
        if possible_positions:
            # Sort positions based on time of day and personal preference
            # This provides a better quality schedule
            def position_score(pos):
                day_preference = {'Senin': 0, 'Selasa': 1, 'Rabu': 2, 'Kamis': 3, 'Jumat': 4}
                day = schedule[pos]['hari']
                time = time_to_minutes(schedule[pos]['jam_mulai'])
                
                # Prefer middle of day (not too early, not too late)
                time_preference = -abs(time - 720)  # Closer to noon is better
                
                return (day_preference.get(day, 5), time_preference)
            
            # Randomly select from top 3 positions if available
            if len(possible_positions) > 3 and random.random() < 0.7:
                possible_positions.sort(key=position_score, reverse=True)
                position = random.choice(possible_positions[:3])
            else:
                position = random.choice(possible_positions)
            
            block = schedule[position:position+sks]
            
            # Update slot dengan informasi kursus
            for slot in block:
                slot.update({
                    "mata_kuliah": mata_kuliah,
                    "dosen": dosen,
                    "kelas": kelas,
                    "sks": sks,
                    "metode": metode
                })
            return True
        
        return False

# Fungsi untuk menjalankan optimasi
def run_gwo_optimization(create_random_schedule_func, calculate_fitness_func, collect_conflicts_func, population_size=10, max_iterations=100):
    gwo = GreyWolfOptimizer(population_size, max_iterations)
    best_solution, best_fitness = gwo.optimize(calculate_fitness_func, create_random_schedule_func, collect_conflicts_func)
    
    return best_solution, best_fitness

# Contoh penggunaan
if __name__ == "__main__":
    best_schedule, best_fitness = run_gwo_optimization(
        create_random_schedule,
        calculate_fitness,
        collect_conflicts,
        population_size=10,
        max_iterations=50
    )
    
    print(f"Optimasi selesai! Fitness terbaik: {best_fitness}")
    
    # Menampilkan jadwal terbaik
    i = 0
    scheduled_courses = set()
    for slot in best_schedule:
        if slot['mata_kuliah'] is not None:
            i += 1
            course_key = (slot['mata_kuliah'], slot['dosen'], slot['kelas'])
            scheduled_courses.add(course_key)
            # print(f"{slot['id_slot']}, {slot['hari']}, {slot['jam_mulai']}-{slot['jam_selesai']}, {slot['ruang']}, {slot['mata_kuliah']}, {slot['dosen']}, {slot['kelas']}")
    
    # Write the best schedule to a JSON file
    with open('output.json', 'w') as f:
        json.dump(best_schedule, f, indent=4)

    target_course_count = None  # Replace with merged_df.shape[0] or appropriate count
    if target_course_count and len(scheduled_courses) == target_course_count:
        print("Jadwal Sudah Lengkap")
    else:
        print("Jadwal Belum Lengkap")
    print(f"Total slot terisi: {i}")
    print(f"Total mata kuliah terjadwal: {len(scheduled_courses)}")

Iterasi 1/50 - Best Fitness: 95.5
Iterasi 2/50 - Best Fitness: 46.0
Iterasi 3/50 - Best Fitness: 46.0
Iterasi 4/50 - Best Fitness: 46.0
Iterasi 5/50 - Best Fitness: 46.0
Iterasi 6/50 - Best Fitness: 46.0
Iterasi 7/50 - Best Fitness: 46.0
Iterasi 8/50 - Best Fitness: 45.0
Iterasi 9/50 - Best Fitness: 45.0
Iterasi 10/50 - Best Fitness: 45.0
Iterasi 11/50 - Best Fitness: 45.0
Iterasi 12/50 - Best Fitness: 45.0
Iterasi 13/50 - Best Fitness: 45.0
Iterasi 14/50 - Best Fitness: 45.0
Iterasi 15/50 - Best Fitness: 45.0
Iterasi 16/50 - Best Fitness: 45.0
Iterasi 17/50 - Best Fitness: 45.0
Iterasi 18/50 - Best Fitness: 45.0
Iterasi 19/50 - Best Fitness: 45.0
Iterasi 20/50 - Best Fitness: 45.0
Iterasi 21/50 - Best Fitness: 45.0
Iterasi 22/50 - Best Fitness: 45.0
Iterasi 23/50 - Best Fitness: 45.0
Iterasi 24/50 - Best Fitness: 45.0
Iterasi 25/50 - Best Fitness: 45.0
Iterasi 26/50 - Best Fitness: 45.0
Iterasi 27/50 - Best Fitness: 45.0
Iterasi 28/50 - Best Fitness: 45.0
Iterasi 29/50 - Best Fitness: